In [7]:
import pandas as pd 
import torch

In [17]:
import pandas as pd
import sys

original_sys_path = sys.path.copy()

sys.path.insert(0, "/Users/florian/Documents/github/study/IoT/IoT/main/")
import dataprep as dp 
import foo

sys.path = original_sys_path

In [26]:
FILENAME = "/Users/florian/Documents/github/study/IoT/IoT/data/aggregated_data/agg_hourly.parquet"
window_size = 50
train_test_split_ratio = 0.8
batch_size = 64

features = ["tmp", "CO2", "hum", "VOC"]
target = "tmp"
scaling = True

In [27]:
data = dp.create_DataLoader(filepath= FILENAME, window_size= window_size, train_ratio= train_test_split_ratio, batch_size= batch_size, features= features, target= target, scaling= scaling)

In [ ]:
input_size = data["train"].x.shape[2]
hidden_sizes = 100
num_layers = 1
output_size = 1
dropouts = 0
activation_functions = "relu"

model = foo.LSTM(input_size= input_size, hidden_size= hidden_sizes, num_layers= num_layers, output_size= output_size, dropout= dropouts, activation= activation_functions)

In [ ]:
from tqdm import tqdm

num_epochs = 10
learning_rate = 0.0001
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
train_loader = data["train"].loader

model.train()
for epoch in range(num_epochs):
    loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=True)
    for i, (features, target) in loop:
        optimizer.zero_grad(set_to_none=True)
        output = model(features)
        #target = target.unsqueeze(1)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        # Update progress bar
        loop.set_description(f"Epoch [{epoch+1}/{num_epochs}]")
        loop.set_postfix(loss=loss.item(), lr= learning_rate)


# Final output
print(f"Training completed. Final loss: {loss.item()}, final learning rate: {learning_rate}")